In [1]:
from dataclasses import dataclass, field
from enum import Enum, auto
import random
import pandas as pd

In [2]:
# cards are numbers from 1 to 13
# the score is capped at 10

In [3]:
@dataclass
class Hand:
    """Class for representing a blackjack hand."""
    score: int = 0
    soft: bool = False
    cards: [int] = field(default_factory=list)


In [4]:
h = Hand(6)
h

Hand(score=6, soft=False, cards=[])

In [5]:
s17 = Hand(17, True)
s17

Hand(score=17, soft=True, cards=[])

In [6]:
# For now, assume aces are always 1
def add_card(hand, card):
    new_score = min(hand.score+min(10, card), 22) # cap busted hands at 22
    hand.score = new_score
    hand.cards += [card]
    return hand

In [7]:
[(c, add_card(Hand(6), c)) for c in range(1,14)]

[(1, Hand(score=7, soft=False, cards=[1])),
 (2, Hand(score=8, soft=False, cards=[2])),
 (3, Hand(score=9, soft=False, cards=[3])),
 (4, Hand(score=10, soft=False, cards=[4])),
 (5, Hand(score=11, soft=False, cards=[5])),
 (6, Hand(score=12, soft=False, cards=[6])),
 (7, Hand(score=13, soft=False, cards=[7])),
 (8, Hand(score=14, soft=False, cards=[8])),
 (9, Hand(score=15, soft=False, cards=[9])),
 (10, Hand(score=16, soft=False, cards=[10])),
 (11, Hand(score=16, soft=False, cards=[11])),
 (12, Hand(score=16, soft=False, cards=[12])),
 (13, Hand(score=16, soft=False, cards=[13]))]

In [8]:
[(c, add_card(Hand(16), c)) for c in range(1,14)]

[(1, Hand(score=17, soft=False, cards=[1])),
 (2, Hand(score=18, soft=False, cards=[2])),
 (3, Hand(score=19, soft=False, cards=[3])),
 (4, Hand(score=20, soft=False, cards=[4])),
 (5, Hand(score=21, soft=False, cards=[5])),
 (6, Hand(score=22, soft=False, cards=[6])),
 (7, Hand(score=22, soft=False, cards=[7])),
 (8, Hand(score=22, soft=False, cards=[8])),
 (9, Hand(score=22, soft=False, cards=[9])),
 (10, Hand(score=22, soft=False, cards=[10])),
 (11, Hand(score=22, soft=False, cards=[11])),
 (12, Hand(score=22, soft=False, cards=[12])),
 (13, Hand(score=22, soft=False, cards=[13]))]

In [9]:
def is_busted(hand):
    return hand.score > 21

In [10]:
[is_busted(add_card(Hand(16), c)) for c in range(1,14)].count(True)

8

In [11]:


class Action(Enum):
    STAND = auto()
    HIT = auto()
    DOUBLE = auto()
    SPLIT = auto()
    
    

In [20]:
# return an Action
def strat_nobust(hand, dealer):
    if hand.score > 11:
        return Action.STAND
    else:
        return Action.HIT
        

In [21]:
# return an Action
def strat_dealer(hand, dealer):
    if hand.score < 17:
        return Action.HIT
    # TODO handle soft hands
    else:
        return Action.STAND
        

In [14]:
class HandOutcome(Enum):
    WIN = 1
    LOSE = -1
    PUSH = 0

In [15]:
# Deck; completely random (i.e., infinite) for now


def deal_card():
    return random.randrange(13)+1

In [16]:
[deal_card() for _ in range(10)]

[2, 13, 6, 1, 12, 8, 10, 7, 6, 8]

In [23]:
# player and dealer play their strategies

# return the final hand after playing
def player_play_hand(strategy, hand, dealer, deck): 
    while True:
        decision = strategy(hand, dealer)
        if decision == Action.STAND:
            return hand
        if decision == Action.HIT:
            add_card(hand, deck())
            if is_busted(hand):
                return hand



hand_p, hand_d

(Hand(score=16, soft=False, cards=[9, 7]),
 Hand(score=22, soft=False, cards=[9, 5, 8]))

In [24]:
def player_hand_outcome(player_hand, dealer_hand):
    # TODO blackjack
    if is_busted(player_hand):
        return HandOutcome.LOSE
    if is_busted(dealer_hand):
        return HandOutcome.WIN
    if player_hand.score > dealer_hand.score:
        return HandOutcome.WIN
    if player_hand.score == dealer_hand.score:
        return HandOutcome.PUSH
    if player_hand.score < dealer_hand.score:
        return HandOutcome.LOSE
    
        

In [51]:
# One player and a dealer
# Player has a strategy
# Each gets dealt cards and plays according to strategy
# Emit all the data, and figure the rest out later

def play_one_hand(strat):
    hand_p = Hand()
    hand_d = Hand()

    add_card(hand_p, deal_card())
    add_card(hand_d, deal_card())
    add_card(hand_p, deal_card())
    dealer_hole_card = deal_card()
    
    # player
    player_play_hand(strat, hand_p, hand_d, deal_card)
    # dealer
    player_play_hand(strat_dealer, add_card(hand_d, dealer_hole_card), None, deal_card)
    
    return (hand_p, hand_d, player_hand_outcome(hand_p, hand_d))


play_one_hand(strat_nobust)

(Hand(score=16, soft=False, cards=[10, 6]),
 Hand(score=17, soft=False, cards=[5, 4, 8]),
 <HandOutcome.LOSE: -1>)

In [52]:

def generate_row_from_hand(h):
    (hand_p, hand_d, outcome) = h
    return {'hand_start': hand_p.cards[:2], 'dealer_card': hand_d.cards[0], 'hand_end': hand_p.cards, 'dealer_hand': hand_d.cards, 'outcome': outcome}

generate_row_from_hand(play_one_hand(strat_nobust))

{'hand_start': [12, 10],
 'dealer_card': 11,
 'hand_end': [12, 10],
 'dealer_hand': [11, 13],
 'outcome': <HandOutcome.PUSH: 0>}

In [56]:
sims = pd.DataFrame([generate_row_from_hand(play_one_hand(strat_dealer)) for _ in range(1000000)])
sims['outcome'].value_counts()

HandOutcome.LOSE    498689
HandOutcome.WIN     402807
HandOutcome.PUSH     98504
Name: outcome, dtype: int64

In [63]:
def run_n_sim_trials(strat, n):
    sims = pd.DataFrame([generate_row_from_hand(play_one_hand(strat)) for _ in range(n)])
    return sims['outcome'].value_counts(), sims['outcome'].apply(lambda x: x.value).mean()

In [61]:
sims = pd.DataFrame([generate_row_from_hand(play_one_hand(strat_nobust)) for _ in range(1000000)])
sims['outcome'].value_counts(), sims['outcome'].apply(lambda x: x.value).mean()

(HandOutcome.LOSE    511949
 HandOutcome.WIN     424909
 HandOutcome.PUSH     63142
 Name: outcome, dtype: int64,
 -0.08704)

In [88]:
def strat_simple(hand, dealer):
    if hand.score >= 17:  return Action.STAND
    if hand.score <= 11:  return Action.HIT
    if dealer.score in (range(3,7)):  return Action.STAND
    else:  return Action.HIT
        
sims = pd.DataFrame([generate_row_from_hand(play_one_hand(strat_simple)) for _ in range(20)])
sims

,hand_start,dealer_card,hand_end,dealer_hand,outcome
0,"[3, 12]",4,"[3, 12]","[4, 1, 7, 13]",HandOutcome.WIN
1,"[10, 8]",4,"[10, 8]","[4, 5, 4, 6]",HandOutcome.LOSE
2,"[10, 10]",8,"[10, 10]","[8, 13]",HandOutcome.WIN
3,"[2, 5]",3,"[2, 5, 2, 7]","[3, 13, 7]",HandOutcome.LOSE
4,"[10, 10]",1,"[10, 10]","[1, 3, 7, 1, 12]",HandOutcome.WIN
5,"[7, 11]",7,"[7, 11]","[7, 4, 11]",HandOutcome.LOSE
6,"[10, 1]",1,"[10, 1, 8]","[1, 3, 9, 4]",HandOutcome.WIN
7,"[11, 13]",12,"[11, 13]","[12, 2, 1, 12]",HandOutcome.WIN
8,"[8, 3]",9,"[8, 3, 9]","[9, 7, 12]",HandOutcome.WIN
9,"[6, 4]",3,"[6, 4, 8]","[3, 6, 4, 6]",HandOutcome.LOSE


In [87]:
for strat in [strat_dealer, strat_nobust, strat_simple]:
    print(run_n_sim_trials(strat, 1000000))

    

(HandOutcome.LOSE    499582
HandOutcome.WIN     401818
HandOutcome.PUSH     98600
Name: outcome, dtype: int64, -0.097764)
(HandOutcome.LOSE    512527
HandOutcome.WIN     424365
HandOutcome.PUSH     63108
Name: outcome, dtype: int64, -0.088162)
(HandOutcome.LOSE    487225
HandOutcome.WIN     423243
HandOutcome.PUSH     89532
Name: outcome, dtype: int64, -0.063982)
